In [1]:
%cd /Users/pg/OneDrive/Duke Assigment/Job Seeking/Collection_DS_takehome_challenges/data
#%cd C:/Users/szdxl/OneDrive/Duke Assigment/Job Seeking/Collection_DS_takehome_challenges/data

/Users/pg/OneDrive/Duke Assigment/Job Seeking/Collection_DS_takehome_challenges/data


In [52]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize

In [3]:
data=pd.read_json('song.json')
data['time_played']=pd.to_datetime(data['time_played'])
data['user_sign_up_date']=pd.to_datetime(data['user_sign_up_date'])

In [4]:
data.head()

,id,song_played,time_played,user_id,user_sign_up_date,user_state
0,GOQMMKSQQH,Hey Jude,2015-06-11 21:51:35,122,2015-05-16,Louisiana
1,HWKKBQKNWI,We Can Work It Out,2015-06-06 16:49:19,3,2015-05-01,Ohio
2,DKQSXVNJDH,Back In the U.S.S.R.,2015-06-14 02:11:29,35,2015-05-04,New Jersey
3,HLHRIDQTUW,P.s. I Love You,2015-06-08 12:26:10,126,2015-05-16,Illinois
4,SUKJCSBCYW,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00,6,2015-05-01,New Jersey


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 6 columns):
id                   4000 non-null object
song_played          4000 non-null object
time_played          4000 non-null datetime64[ns]
user_id              4000 non-null int64
user_sign_up_date    4000 non-null datetime64[ns]
user_state           4000 non-null object
dtypes: datetime64[ns](2), int64(1), object(3)
memory usage: 187.6+ KB


**Question 1:What are the top 3 and the bottom 3 states in terms of number of users?**

In [6]:
data.head()

,id,song_played,time_played,user_id,user_sign_up_date,user_state
0,GOQMMKSQQH,Hey Jude,2015-06-11 21:51:35,122,2015-05-16,Louisiana
1,HWKKBQKNWI,We Can Work It Out,2015-06-06 16:49:19,3,2015-05-01,Ohio
2,DKQSXVNJDH,Back In the U.S.S.R.,2015-06-14 02:11:29,35,2015-05-04,New Jersey
3,HLHRIDQTUW,P.s. I Love You,2015-06-08 12:26:10,126,2015-05-16,Illinois
4,SUKJCSBCYW,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00,6,2015-05-01,New Jersey


In [7]:
sort=data[['user_id','user_state']].groupby('user_state').nunique().sort_values('user_id', ascending = False)[['user_id']]
sort.rename(columns={'user_id': 'count'}, inplace=True)

In [8]:
sort.head(3)
#states with most count

,count
user_state,
New York,23
California,21
Texas,15


In [9]:
sort.tail(9)
#states with least countsm

,count
user_state,
New Mexico,1
Idaho,1
North Dakota,1
Connecticut,1
Iowa,1
Rhode Island,1
Nebraska,1
Arizona,1
Kansas,1


**What are the top 3 and the bottom 3 states in terms of user engagement? You can choose how to mathematically define user engagement. What the CEO cares about here is in which states users are using the product a lot/very little.**

In [10]:
data.head()

,id,song_played,time_played,user_id,user_sign_up_date,user_state
0,GOQMMKSQQH,Hey Jude,2015-06-11 21:51:35,122,2015-05-16,Louisiana
1,HWKKBQKNWI,We Can Work It Out,2015-06-06 16:49:19,3,2015-05-01,Ohio
2,DKQSXVNJDH,Back In the U.S.S.R.,2015-06-14 02:11:29,35,2015-05-04,New Jersey
3,HLHRIDQTUW,P.s. I Love You,2015-06-08 12:26:10,126,2015-05-16,Illinois
4,SUKJCSBCYW,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00,6,2015-05-01,New Jersey


In [11]:
play_count = data.groupby('user_state')[['id']].count().reset_index()
play_count = play_count.rename(columns={'id': 'play_count'})
state_use = pd.merge(left=sort, right = play_count, on='user_state')

In [12]:
state_use['average_use'] = state_use['play_count']/state_use['count']
state_use.sort_values(by='average_use', inplace=True, ascending = False)
print('most engagement:\n', state_use.head(3))
print('\n')
print('least engagement:\n', state_use.tail(3))

most engagement:
         user_state  count  play_count  average_use
38        Nebraska      1          36    36.000000
27          Alaska      2          58    29.000000
22  South Carolina      3          85    28.333333


least engagement:
    user_state  count  play_count  average_use
17  Minnesota      4          42         10.5
30   Virginia      2          17          8.5
40     Kansas      1           8          8.0


**Q3: The CEO wants to send a gift to the first user who signed-up for each state. That is, the
first user who signed-up from California, from Oregon, etc. Can you give him a list of
those users?**

In [13]:
data.head()

,id,song_played,time_played,user_id,user_sign_up_date,user_state
0,GOQMMKSQQH,Hey Jude,2015-06-11 21:51:35,122,2015-05-16,Louisiana
1,HWKKBQKNWI,We Can Work It Out,2015-06-06 16:49:19,3,2015-05-01,Ohio
2,DKQSXVNJDH,Back In the U.S.S.R.,2015-06-14 02:11:29,35,2015-05-04,New Jersey
3,HLHRIDQTUW,P.s. I Love You,2015-06-08 12:26:10,126,2015-05-16,Illinois
4,SUKJCSBCYW,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00,6,2015-05-01,New Jersey


In [14]:
def find_first_user(df):
    idx=df['user_sign_up_date'].idxmin()
    return df.loc[idx, ['user_id', 'user_sign_up_date']]

In [15]:
data.groupby('user_state').apply(find_first_user).sort_values(by='user_sign_up_date')

,user_id,user_sign_up_date
user_state,,
Alabama,5,2015-05-01
Texas,7,2015-05-01
Oregon,1,2015-05-01
Ohio,3,2015-05-01
North Carolina,2,2015-05-01
New Mexico,4,2015-05-01
New Jersey,6,2015-05-01
Pennsylvania,11,2015-05-02
New York,19,2015-05-02


**Q4: Build a function that takes as an input any of the songs in the data and returns the most
likely song to be listened next. That is, if, for instance, a user is currently listening to
"Eight Days A Week", which song has the highest probability of being played right after it
by the same user? This is going to be v1 of a song recommendation model.**

In [16]:
data['user_id'].nunique()

196

In [57]:
song_user = data.groupby(['song_played', 'user_id'])['id'].count().unstack(fill_value=0)
song_user = (song_user > 0).astype(int)
song_user_norm=normalize(song_user, axis=1)
sim = np.dot(song_user_norm, song_user_norm.T)
sim_df = pd.DataFrame(sim, index = song_user.index, columns=song_user.index)
sim_df.head()

song_played,A Day In The Life,A Hard Day's Night,A Saturday Club Xmas/Crimble Medley,ANYTIME AT ALL,Across The Universe,All My Loving,All You Need Is Love,And Your Bird Can Sing,BAD BOY,BALLAD OF JOHN AND YOKO,...,We Can Work It Out,When I'm 64,While My Guitar Gently Weeps,Wild Honey Pie,With a Little Help From My Friends,YOUR MOTHER SHOULD KNOW,Yellow Submarine,Yesterday,You Never Give Me Your Money,You're Going To Lose That Girl
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,1.000000,0.264392,0.139347,0.148968,0.132196,0.301023,0.295599,0.098533,0.197066,0.201129,...,0.516528,0.056888,0.578459,0.279852,0.399723,0.088131,0.330489,0.365433,0.164222,0.0
A Hard Day's Night,0.264392,1.000000,0.000000,0.000000,0.100000,0.146385,0.111803,0.000000,0.000000,0.091287,...,0.305788,0.129099,0.266996,0.000000,0.000000,0.000000,0.050000,0.215003,0.074536,0.0
A Saturday Club Xmas/Crimble Medley,0.139347,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.161165,0.000000,0.000000,0.182574,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
ANYTIME AT ALL,0.148968,0.000000,0.000000,1.000000,0.000000,0.164957,0.094491,0.125988,0.000000,0.000000,...,0.172292,0.000000,0.188044,0.097590,0.191663,0.000000,0.000000,0.103835,0.000000,0.0
Across The Universe,0.132196,0.100000,0.000000,0.000000,1.000000,0.097590,0.000000,0.000000,0.000000,0.000000,...,0.101929,0.000000,0.133498,0.000000,0.000000,0.000000,0.000000,0.061430,0.000000,0.0


In [86]:
def find_k_song(song, k):
    df = sim_df.loc[song, :].sort_values(ascending = False)[1:k+1].index.values
    return print(df)

In [87]:
find_k_song('Eight Days A Week', 10)
#so here is the recommendation system

['Come Together' 'Hey Jude' 'While My Guitar Gently Weeps' 'Penny Lane'
 'A Day In The Life' 'Let It Be' 'Get Back' 'Revolution'
 "I've Got A Feeling" 'We Can Work It Out']


**How would you set up a test to check whether your model works well and is improving engagement?**

We need to design an A/B test

1.Randomly split the users into two groups, one doesn't apply recommendation system while the other was applied

2.After running some time, do the test. An one-tailed t_test could be performed on metrics such as #of hours listening or # of songs continue to listen

